<a href="https://colab.research.google.com/github/castudil/bacteria-multi-label/blob/main/multilabel_bac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries used

In [1]:
import os
import copy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (f1_score, multilabel_confusion_matrix,
                             accuracy_score, hamming_loss, jaccard_score, make_scorer)
from sklearn.model_selection import train_test_split, RepeatedKFold

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

import keras_tuner as kt
import keras

from joblib import dump, load
import joblib

2023-12-12 16:03:42.059685: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-12 16:03:43.517885: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 16:03:43.517946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 16:03:43.582324: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 16:03:43.883517: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tf.random.set_seed(0)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Data Loading

In [3]:
os.chdir("..")

In [4]:
train_file = "data/processed/raw/train_s_aureus_driams.csv"
train_bac = pd.read_csv(train_file)
train_bac

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,9993,9994,9995,9996,9997,9998,9999,Oxacillin,Clindamycin,Fusidic acid
0,0.018721,0.016147,0.016983,0.021218,0.020846,0.019784,0.019405,0.023356,0.026224,0.026569,...,0.037966,0.030364,0.037545,0.040851,0.034176,0.046110,0.025638,0.0,0.0,0.0
1,0.009001,0.007475,0.006874,0.008575,0.009539,0.007894,0.008314,0.008013,0.008664,0.008923,...,0.014496,0.024966,0.027437,0.026541,0.022940,0.020572,0.032504,0.0,0.0,0.0
2,0.022354,0.020220,0.020910,0.024631,0.021436,0.021197,0.020229,0.018818,0.018637,0.018815,...,0.024620,0.022942,0.026715,0.032045,0.030431,0.029085,0.013117,0.0,0.0,0.0
3,0.017619,0.016073,0.016407,0.018011,0.019364,0.018950,0.017607,0.019116,0.023623,0.024492,...,0.051312,0.047458,0.049338,0.055039,0.054541,0.058643,0.058919,0.0,0.0,0.0
4,0.008264,0.008229,0.006753,0.006657,0.010107,0.007039,0.008250,0.010670,0.008134,0.006513,...,0.236769,0.217499,0.187244,0.216243,0.221910,0.226531,0.221965,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2819,0.056616,0.039011,0.040380,0.048517,0.050865,0.047771,0.049312,0.048257,0.049417,0.049000,...,0.021169,0.023617,0.033694,0.021037,0.018727,0.010641,0.009238,0.0,1.0,0.0
2820,0.125837,0.107712,0.109186,0.107613,0.109855,0.105060,0.099640,0.104169,0.120303,0.125067,...,0.082375,0.083446,0.096510,0.084883,0.092228,0.085599,0.042142,0.0,1.0,0.0
2821,0.000000,0.000000,0.035603,0.039994,0.042372,0.046666,0.045781,0.043914,0.039875,0.037170,...,0.006903,0.008322,0.011071,0.010274,0.004682,0.003547,0.001744,0.0,0.0,0.0
2822,0.005443,0.005998,0.003670,0.005588,0.006124,0.005019,0.004853,0.005400,0.004169,0.005151,...,0.049241,0.039586,0.050542,0.039139,0.046816,0.043036,0.037402,1.0,1.0,0.0


In [5]:
test_file = "data/processed/raw/test_s_aureus_driams.csv"
test_bac = pd.read_csv(test_file)
test_bac

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,9993,9994,9995,9996,9997,9998,9999,Oxacillin,Clindamycin,Fusidic acid
0,0.044453,0.032486,0.032540,0.034223,0.037528,0.039503,0.031378,0.035506,0.037688,0.035658,...,0.247814,0.263833,0.279904,0.264432,0.241573,0.266020,0.231517,0.0,0.0,0.0
1,0.004318,0.001881,0.001274,0.000902,0.000892,0.000049,0.002188,0.002001,0.003081,0.003384,...,0.033364,0.042735,0.066426,0.057485,0.052903,0.050839,0.036631,0.0,0.0,0.0
2,0.026184,0.026459,0.025393,0.028609,0.031314,0.031739,0.033337,0.028051,0.028047,0.028978,...,0.086746,0.087719,0.094103,0.080969,0.073970,0.069047,0.070988,0.0,1.0,0.0
3,0.000000,0.015010,0.017782,0.014582,0.015084,0.018046,0.014461,0.014400,0.018769,0.018272,...,0.005062,0.006748,0.004573,0.007583,0.008427,0.004729,0.002394,0.0,0.0,1.0
4,0.060499,0.043034,0.030296,0.032635,0.031272,0.032762,0.031154,0.030382,0.030233,0.029438,...,0.029452,0.033288,0.039230,0.046477,0.037219,0.022227,0.019920,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,0.013092,0.011730,0.009902,0.013513,0.015317,0.011370,0.013338,0.010117,0.010994,0.009222,...,0.050851,0.046334,0.042840,0.061644,0.066948,0.061007,0.056178,0.0,0.0,0.0
703,0.021222,0.015896,0.015512,0.017995,0.018663,0.019427,0.018667,0.014589,0.016765,0.015071,...,0.047285,0.056455,0.058002,0.055528,0.043539,0.039962,0.034761,1.0,0.0,0.0
704,0.045613,0.041040,0.046177,0.050216,0.046525,0.048843,0.045920,0.044283,0.043983,0.046330,...,0.104234,0.094692,0.090012,0.096624,0.092228,0.100024,0.043682,0.0,0.0,1.0
705,0.015193,0.011922,0.010877,0.009975,0.010474,0.012171,0.010417,0.010783,0.013170,0.014157,...,0.051772,0.058255,0.074609,0.068249,0.049157,0.070229,0.043615,1.0,0.0,0.0


In [6]:
train_x = train_bac[train_bac.columns.drop(list(train_bac.filter(regex='[^0-9]')))]
test_x = test_bac[test_bac.columns.drop(list(test_bac.filter(regex='[^0-9]')))]

In [7]:
antibiotics = train_bac.columns.drop(train_x.columns)

In [8]:
train_y = train_bac[antibiotics]
test_y = test_bac[antibiotics]

In [9]:
with open("data/features/s_aureus_driams_selected_features.txt") as file:
    selected_features = file.read().split(",")
selected_features.pop()

train_x = train_x[selected_features]
test_x = test_x[selected_features]

# Evaluation

In [10]:
def multilabel_f1_wrapper(true, pred, average="weighted"):
    if isinstance(true, list):
        true = np.array(true)
    elif isinstance(true, pd.DataFrame):
        true = true.to_numpy()
    if isinstance(pred, list):
        pred = np.array(pred)
    elif isinstance(true, pd.DataFrame):
        pred = pred.to_numpy()
    column = 0
    total = 0
    while column < true[0].size:
        total+=f1_score(true[:, column], pred[:, column], average=average)
        column+=1
    return total/(column)

In [11]:
def report(true, pred):
        
    hl = hamming_loss(true, pred)
    f1w = multilabel_f1_wrapper(true, pred, "weighted")
    acc = accuracy_score(true, pred)
    
    f1u = multilabel_f1_wrapper(true, pred, "macro")
    f1su = f1_score(true, pred, average="macro")
    f1sw = f1_score(true, pred, average="weighted")

    
    print("Main metrics:")
    print(" Hamming Loss:", hl)
    print(" Accuracy:", acc)
    print(" F1 Score (Weighted):", f1w)
    print("================================================")
    print("Other metrics:")
    print(" F1 Score (Unweighted):", f1u)
    print(" F1 Score (sklearn Unweighted):", f1su)
    print(" F1 Score (sklearn Weighted):", f1sw)
    return hl, acc, f1w

# Model

In [12]:
class BaseBacModel(Model):
  def __init__(self, input_dim, output_dim, units):
    super().__init__()
    self.d1 = Dense(units, input_dim=input_dim, kernel_initializer="he_uniform", activation="relu")
    self.d2 = Dense(output_dim, activation='sigmoid')

  def call(self, x):
    x = self.d1(x)
    return self.d2(x)

In [13]:
def evaluate_model(x, y):
    results = list()
    
    cv = RepeatedKFold(n_splits=5, n_repeats=1, random_state=1)
    
    np_x = x.to_numpy()
    np_y = y.to_numpy()
    
    for train_i, validation_i in cv.split(np_x):
        input_dim  = len(x.columns)
        output_dim = len(y.columns)
        model = BaseBacModel(input_dim, output_dim, 512)
        model.compile(loss="binary_crossentropy", optimizer="adam")
        
        train_x = np_x[train_i]
        train_y = np_y[train_i]
        validation_x = np_x[validation_i]
        validation_y = np_y[validation_i]

        model.fit(train_x, train_y, verbose=1, epochs=100)

        pred = model.predict(validation_x)
        pred = pred.round()
        
        hl, acc, f1w = report(validation_y, pred)

        results.append(f1w)
    return results

In [14]:
evaluate_model(train_x, train_y)

Epoch 1/100


2023-12-12 16:03:57.690060: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-12-12 16:03:57.690119: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: dslab
2023-12-12 16:03:57.690129: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: dslab
2023-12-12 16:03:57.690325: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 470.199.2
2023-12-12 16:03:57.690357: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 470.199.2
2023-12-12 16:03:57.690365: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 470.199.2


71/71 [==============================] - 1s 3ms/step - loss: 0.4572
Epoch 2/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3848
Epoch 3/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3569
Epoch 4/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3435
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3372
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3308
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3283
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3232
Epoch 9/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3186
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3162
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3119
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3096
Epoch 13/100
71/71 [=============================

Epoch 1/100
71/71 [==============================] - 0s 2ms/step - loss: 0.4606
Epoch 2/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3823
Epoch 3/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3570
Epoch 4/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3453
Epoch 5/100
71/71 [==============================] - 0s 3ms/step - loss: 0.3371
Epoch 6/100
71/71 [==============================] - 0s 3ms/step - loss: 0.3330
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3290
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3230
Epoch 9/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3217
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3148
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3128
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3090
Epoch 13/100
71/71 [=================

Epoch 1/100
71/71 [==============================] - 0s 2ms/step - loss: 0.4539
Epoch 2/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3764
Epoch 3/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3488
Epoch 4/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3380
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3307
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3249
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3199
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3207
Epoch 9/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3120
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3086
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3054
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3028
Epoch 13/100
71/71 [=================

Epoch 1/100
71/71 [==============================] - 0s 2ms/step - loss: 0.4610
Epoch 2/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3920
Epoch 3/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3646
Epoch 4/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3520
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3435
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3410
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3334
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3284
Epoch 9/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3247
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3205
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3192
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3150
Epoch 13/100
71/71 [=================

Epoch 1/100
71/71 [==============================] - 0s 2ms/step - loss: 0.4574
Epoch 2/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3866
Epoch 3/100
71/71 [==============================] - 0s 3ms/step - loss: 0.3561
Epoch 4/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3448
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3370
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3329
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3271
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3234
Epoch 9/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3187
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3143
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3139
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3088
Epoch 13/100
71/71 [=================

[0.886130412910842,
 0.8800419848643405,
 0.856611204697506,
 0.8925444105143705,
 0.8722915379188297]

In [15]:
def cross_train(x, y, model):
    results = list()
    
    cv = RepeatedKFold(n_splits=5, n_repeats=1, random_state=1)
    
    np_x = x.to_numpy()
    np_y = y.to_numpy()
    
    for train_i, validation_i in cv.split(np_x):
        train_x = np_x[train_i]
        train_y = np_y[train_i]
        validation_x = np_x[validation_i]
        validation_y = np_y[validation_i]

        model.fit(train_x, train_y, verbose=1, epochs=100)

        pred = model.predict(validation_x)
        pred = pred.round()
        
        hl, acc, f1w = report(validation_y, pred)

        results.append(f1w)
    return results

In [16]:
input_dim  = len(train_x.columns)
output_dim = len(train_y.columns)
model = BaseBacModel(input_dim, output_dim, 512)
model.compile(loss="binary_crossentropy", optimizer="adam")
cross_train(train_x, train_y, model)

Epoch 1/100
71/71 [==============================] - 0s 2ms/step - loss: 0.4558
Epoch 2/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3859
Epoch 3/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3578
Epoch 4/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3441
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3375
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3309
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3284
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3231
Epoch 9/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3184
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3160
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3116
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3089
Epoch 13/100
71/71 [=================

Epoch 1/100
71/71 [==============================] - 0s 1ms/step - loss: 0.2231
Epoch 2/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2214
Epoch 3/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2174
Epoch 4/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2168
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2129
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2127
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2127
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2159
Epoch 9/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2094
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2079
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2070
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2048
Epoch 13/100
71/71 [=================

Epoch 1/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1633
Epoch 2/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1590
Epoch 3/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1594
Epoch 4/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1567
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1536
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1522
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1516
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1541
Epoch 9/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1513
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1468
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1457
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1450
Epoch 13/100
71/71 [=================

Epoch 1/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1338
Epoch 2/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1295
Epoch 3/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1288
Epoch 4/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1254
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1222
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1216
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1227
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1206
Epoch 9/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1190
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1170
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1183
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1161
Epoch 13/100
71/71 [=================

Epoch 1/100
71/71 [==============================] - 0s 1ms/step - loss: 0.0990
Epoch 2/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0940
Epoch 3/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0905
Epoch 4/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0906
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0875
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0891
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0873
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0875
Epoch 9/100
71/71 [==============================] - 0s 3ms/step - loss: 0.0853
Epoch 10/100
71/71 [==============================] - 0s 3ms/step - loss: 0.0837
Epoch 11/100
71/71 [==============================] - 0s 3ms/step - loss: 0.0835
Epoch 12/100
71/71 [==============================] - 0s 3ms/step - loss: 0.0820
Epoch 13/100
71/71 [=================

[0.8865277054527382,
 0.9013509414862919,
 0.903953860499588,
 0.943258154902156,
 0.9454934438149419]

In [17]:
pred = model.predict(test_x)
pred_binary = pred.round()

hl, acc, f1w = report(test_y, pred_binary)

23/23 [==============================] - 0s 640us/step
Main metrics:
 Hamming Loss: 0.10985384252710985
 Accuracy: 0.7199434229137199
 F1 Score (Weighted): 0.8768480826575886
Other metrics:
 F1 Score (Unweighted): 0.6780018339784029
 F1 Score (sklearn Unweighted): 0.41973046477353876
 F1 Score (sklearn Weighted): 0.4921601590821736


# Optimization

In [18]:
def model_builder(hp):
    
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    
    input_dim  = len(train_x.columns)
    output_dim = len(train_y.columns)
    
    model = BaseBacModel(input_dim, output_dim, hp_units)
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss="binary_crossentropy",
    )

    return model

In [19]:
tuner = kt.BayesianOptimization(
    model_builder,
    objective="val_loss",
    max_trials=300,
)

In [20]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [21]:
tuner.search(train_x, train_y, epochs=150, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


Trial 300 Complete [00h 00m 04s]
val_loss: 0.310035765171051

Best val_loss So Far: 0.3031507134437561
Total elapsed time: 00h 26m 51s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 160 and the optimal learning rate for the optimizer
is 0.01.



In [22]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_x, train_y, epochs=150, validation_split=0.2)

val_acc_per_epoch = history.history["val_loss"]
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))


Epoch 1/150
71/71 [==============================] - 1s 3ms/step - loss: 0.4000 - val_loss: 0.3550
Epoch 2/150
71/71 [==============================] - 0s 2ms/step - loss: 0.3523 - val_loss: 0.3442
Epoch 3/150
71/71 [==============================] - 0s 1ms/step - loss: 0.3351 - val_loss: 0.3331
Epoch 4/150
71/71 [==============================] - 0s 2ms/step - loss: 0.3251 - val_loss: 0.3277
Epoch 5/150
71/71 [==============================] - 0s 2ms/step - loss: 0.3174 - val_loss: 0.3288
Epoch 6/150
71/71 [==============================] - 0s 2ms/step - loss: 0.3161 - val_loss: 0.3244
Epoch 7/150
71/71 [==============================] - 0s 1ms/step - loss: 0.3072 - val_loss: 0.3427
Epoch 8/150
71/71 [==============================] - 0s 1ms/step - loss: 0.3042 - val_loss: 0.3302
Epoch 9/150
71/71 [==============================] - 0s 2ms/step - loss: 0.3042 - val_loss: 0.3222
Epoch 10/150
71/71 [==============================] - 0s 2ms/step - loss: 0.2972 - val_loss: 0.3199
Epoch 11/

71/71 [==============================] - 0s 2ms/step - loss: 0.1451 - val_loss: 0.4062
Epoch 84/150
71/71 [==============================] - 0s 1ms/step - loss: 0.1450 - val_loss: 0.3963
Epoch 85/150
71/71 [==============================] - 0s 1ms/step - loss: 0.1467 - val_loss: 0.4104
Epoch 86/150
71/71 [==============================] - 0s 2ms/step - loss: 0.1424 - val_loss: 0.4148
Epoch 87/150
71/71 [==============================] - 0s 2ms/step - loss: 0.1428 - val_loss: 0.4089
Epoch 88/150
71/71 [==============================] - 0s 2ms/step - loss: 0.1389 - val_loss: 0.4073
Epoch 89/150
71/71 [==============================] - 0s 2ms/step - loss: 0.1372 - val_loss: 0.4200
Epoch 90/150
71/71 [==============================] - 0s 2ms/step - loss: 0.1374 - val_loss: 0.4288
Epoch 91/150
71/71 [==============================] - 0s 2ms/step - loss: 0.1346 - val_loss: 0.4173
Epoch 92/150
71/71 [==============================] - 0s 2ms/step - loss: 0.1354 - val_loss: 0.4258
Epoch 93/150


In [23]:
model = tuner.hypermodel.build(best_hps)

# Retrain the model
cross_train(train_x, train_y, model)

Epoch 1/100
71/71 [==============================] - 0s 1ms/step - loss: 0.4014
Epoch 2/100
71/71 [==============================] - 0s 1ms/step - loss: 0.3492
Epoch 3/100
71/71 [==============================] - 0s 1ms/step - loss: 0.3343
Epoch 4/100
71/71 [==============================] - 0s 1ms/step - loss: 0.3218
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3269
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3103
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3090
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.3013
Epoch 9/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2982
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2976
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2912
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.2881
Epoch 13/100
71/71 [=================

Epoch 1/100
71/71 [==============================] - 0s 934us/step - loss: 0.2096
Epoch 2/100
71/71 [==============================] - 0s 1ms/step - loss: 0.2047
Epoch 3/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1909
Epoch 4/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1880
Epoch 5/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1785
Epoch 6/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1776
Epoch 7/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1754
Epoch 8/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1649
Epoch 9/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1637
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1596
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1610
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1519
Epoch 13/100
71/71 [===============

71/71 [==============================] - 0s 932us/step - loss: 0.1946
Epoch 2/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1837
Epoch 3/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1399
Epoch 4/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1324
Epoch 5/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1177
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1083
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0995
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1000
Epoch 9/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0972
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1135
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0944
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0876
Epoch 13/100
71/71 [===========================

71/71 [==============================] - 0s 940us/step - loss: 0.1470
Epoch 2/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1726
Epoch 3/100
71/71 [==============================] - 0s 2ms/step - loss: 0.1287
Epoch 4/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0935
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0734
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0701
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0796
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0761
Epoch 9/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0663
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0637
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0716
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0593
Epoch 13/100
71/71 [===========================

Epoch 1/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1312
Epoch 2/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1127
Epoch 3/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0702
Epoch 4/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0554
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0471
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0456
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0563
Epoch 8/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0424
Epoch 9/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0394
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0331
Epoch 11/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0335
Epoch 12/100
71/71 [==============================] - 0s 2ms/step - loss: 0.0458
Epoch 13/100
71/71 [=================

[0.8797605319726202,
 0.8996186260641709,
 0.9095103528715557,
 0.9522495197575852,
 0.9484634977795269]

In [24]:
pred = model.predict(test_x)
pred_binary = pred.round()

hl, acc, f1w = report(test_y, pred_binary)

23/23 [==============================] - 0s 644us/step
Main metrics:
 Hamming Loss: 0.132013201320132
 Accuracy: 0.6633663366336634
 F1 Score (Weighted): 0.8636790739870047
Other metrics:
 F1 Score (Unweighted): 0.6683956836802952
 F1 Score (sklearn Unweighted): 0.41447337221985103
 F1 Score (sklearn Weighted): 0.4829331839362714
